In [ ]:
import tensorflow as tf
import numpy as np

# Local imports
from train_estimator import IMAGE_FILENAMES, MASK_FILENAMES, \
    NUM_FOLDS, MODEL_DIR
from KFolds import KFolds
from model import model_fn
from input_fn import input_fn

FOLD_NUM = 0
NUM_RANDOM_IMAGES = 5
folds = KFolds(IMAGE_FILENAMES, MASK_FILENAMES,
    num_folds=NUM_FOLDS, sort=False, yield_dict=False)

(_, _), (eval_images, eval_masks) = folds.get_fold(FOLD_NUM)

# TODO: Consider importing parameters from `train_estimator.py`
# This would require moving it out of main and into global scope.
params = {
    'data_format': 'channels_first',
    'num_output_classes': 2,
    'learning_rate': 1e-4
}

image_segmentor = tf.estimator.Estimator(
    model_dir='-'.join([MODEL_DIR, str(FOLD_NUM)]),
    model_fn=model_fn,
    params=params)

In [ ]:
# Choose some random cars from the evaluation set.
random_indexes = np.random.choice(len(eval_images), size=NUM_RANDOM_IMAGES, replace=False)
eval_images = np.array(eval_images) # Convert to np.array to allow for
eval_masks = np.array(eval_masks)   # list slicing.

predictions = image_segmentor.predict(
    input_fn=lambda: input_fn(
        eval_images[random_indexes],
        eval_masks[random_indexes],
        training=False,
        data_format=params['data_format']))

In [ ]:
images = []
masks = []
unconfidence = []

for prediction in predictions:
    images.append(prediction['images'])
    masks.append(prediction['masks'])
    unconfidence.append(prediction['unconfidence'])

In [ ]:
print(images[0].shape)
print(masks[0].shape)
print(unconfidence[0].shape)

In [ ]:
# Rearrange RGB image to channels_last, if necessary
if params['data_format'] == 'channels_first':
    for i, image in enumerate(images):
        r_channel = image[0, :, :]
        g_channel = image[1, :, :]
        b_channel = image[2, :, :]
        images[i] = np.stack([r_channel, g_channel, b_channel], axis=-1)

In [ ]:
print(images[0].shape)
print(masks[0].shape)
print(unconfidence[0].shape)

In [ ]:
# Note that images and predictions need to be in [height, width, channel]
# format in order to be displayed using plt.imshow
from matplotlib import pyplot as plt
% matplotlib inline

In [ ]:
fig, axes = plt.subplots(1, 3)
fig.set_size_inches(h=6, w=18)

index = 2

axes[0].imshow(images[index])
axes[0].set_title('Original image')
axes[1].imshow(masks[index])
axes[1].set_title('Predicted mask')
axes[2].imshow(unconfidence[index])
axes[2].set_title('Unconfidence score')

for ax in axes:
    ax.set_xticks([])
    ax.set_yticks([])

# plt.savefig('images/unconfidence.png')

In [ ]:
from PIL import Image

In [ ]:
im = unconfidence[index] * 255
im = im.astype(np.uint8)
im = Image.fromarray(im)
im.save('images/unconfidence_{}.png'.format(index))

In [ ]:
im = images[index] * 255
im = im.astype(np.uint8)
im = Image.fromarray(im)
im.save('images/original_{}.png'.format(index))